In [25]:
import os 
import json 
import pandas as pd 
import numpy as np
from itertools import chain 

In [2]:
prod_path = "../annotations/prod"
os.listdir(prod_path)

['factscore.jsonl', 'cliff.jsonl']

# load cliff data

In [3]:
df_cliff = pd.read_json(os.path.join(prod_path, "cliff.jsonl"), lines=True)
df_cliff.head()

,source_id,source,summary,cliff_labels,model,datasource,dataset,qas,spans
0,132,"[Media, playback, is, not, supported, on, this...","[[Laura, Muir, says, she, is, "", gutted, "", sh...","[[world knowledge, correct, correct, correct, ...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [5, ..."
1,132,"[Media, playback, is, not, supported, on, this...","[[Laura, Muir, says, she, is, "", gutted, "", sh...","[[world knowledge, correct, correct, correct, ...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [5, ..."
2,58,"[Trevor, Deely, ,, 22, ,, was, last, seen, wal...","[[Gardaí, (, Irish, police, ), investigating, ...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 5...","[{'id': 0, 'start': 2, 'end': 4, 'qaIds': [7, ..."
3,58,"[Trevor, Deely, ,, 22, ,, was, last, seen, wal...","[[Police, in, the, Republic, of, Ireland, have...","[[correct, correct, correct, correct, correct,...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 1, 'start': 3, 'end': 6, 'qaIds': [10,..."
4,20,"[The, boy, ,, Julian, Hernandez, ,, disappeare...","[[The, father, of, an, Alabama, boy, who, went...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 2, 'start': 4, 'end': 5, 'qaIds': [8],..."


In [6]:
df_cliff["qa_labels"] = df_cliff["qas"].apply(lambda lst: [1 if sum(qa["annotations"]) <= 1 else 0 for qa in lst])
df_cliff["qasem_score"] = df_cliff["qa_labels"].apply(np.average) 

In [7]:
df_cliff.head()

,source_id,source,summary,cliff_labels,model,datasource,dataset,qas,spans,qa_labels,qasem_score
0,132,"[Media, playback, is, not, supported, on, this...","[[Laura, Muir, says, she, is, "", gutted, "", sh...","[[world knowledge, correct, correct, correct, ...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [5, ...","[0, 1, 0, 0, 1, 0, 1, 0, 0, 0]",0.300000
1,132,"[Media, playback, is, not, supported, on, this...","[[Laura, Muir, says, she, is, "", gutted, "", sh...","[[world knowledge, correct, correct, correct, ...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [5, ...","[0, 1, 0, 0, 1, 0, 0]",0.285714
2,58,"[Trevor, Deely, ,, 22, ,, was, last, seen, wal...","[[Gardaí, (, Irish, police, ), investigating, ...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 5...","[{'id': 0, 'start': 2, 'end': 4, 'qaIds': [7, ...","[1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]",0.428571
3,58,"[Trevor, Deely, ,, 22, ,, was, last, seen, wal...","[[Police, in, the, Republic, of, Ireland, have...","[[correct, correct, correct, correct, correct,...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 1, 'start': 3, 'end': 6, 'qaIds': [10,...","[1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0]",0.545455
4,20,"[The, boy, ,, Julian, Hernandez, ,, disappeare...","[[The, father, of, an, Alabama, boy, who, went...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 2, 'start': 4, 'end': 5, 'qaIds': [8],...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1.000000


In [8]:
df_cliff["cliff_score"] = df_cliff["cliff_labels"].apply(lambda lst: np.average([1 if x == "correct" else 0 for x in lst[0]]))

In [12]:
# Pivot the DataFrame
pivoted = df_cliff.pivot(index='source_id', columns='model', values=['qasem_score', 'cliff_score'])

# Compute the differences
pivoted['diff_qasem_score'] = pivoted[('qasem_score', 'bart')] - pivoted[('qasem_score', 'pegasus')]
pivoted['diff_cliff_score'] = pivoted[('cliff_score', 'bart')] - pivoted[('cliff_score', 'pegasus')]

# Reset the index and keep only the required columns
result = pivoted[['diff_qasem_score', 'diff_cliff_score']].reset_index()

In [13]:
result

,source_id,diff_qasem_score,diff_cliff_score
model,,,
0,0,0.058824,-0.136775
1,2,0.214286,0.035256
2,3,0.114286,0.000000
3,6,-0.333333,0.000000
4,8,-0.121429,-0.008772
5,17,-0.363636,-0.269231
6,20,0.000000,0.000000
7,22,0.409091,-0.026738
8,27,-0.089286,-0.080000


# load factscore data 

In [14]:
df_factscore = pd.read_json(os.path.join(prod_path, "factscore.jsonl"), lines=True)
df_factscore.head()

,source_id,source,summary,model,datasource,dataset,qas,spans
0,"William Waldegrave, Baron Waldegrave of North ...","[William, Waldegrave, ,, Baron, Waldegrave, of...","[[William, Waldegrave, ,, Baron, Waldegrave, o...",InstructGPT,open_ended,factscore,"[{'qa_id': 0, 'sent_id': 1, 'predicate_idx': 1...","[{'id': 0, 'start': 15, 'end': 16, 'qaIds': [0..."
1,"William Waldegrave, Baron Waldegrave of North ...","[William, Waldegrave, ,, Baron, Waldegrave, of...","[[William, Arthur, Waldegrave, ,, Baron, Walde...",PerplexityAI,open_ended,factscore,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 1...","[{'id': 0, 'start': 0, 'end': 11, 'qaIds': [0]..."
2,Mike Trivisonno,"[Mike, Trivisonno, Mike, Trivisonno, (, Septem...","[[Mike, Trivisonno, is, a, well, -, known, rad...",ChatGPT,open_ended,factscore,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 6...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [0],..."
3,Mike Trivisonno,"[Mike, Trivisonno, Mike, Trivisonno, (, Septem...","[[Mike, Trivisonno, is, a, popular, radio, tal...",InstructGPT,open_ended,factscore,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 8...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [0],..."
4,Andrew Shue,"[Andrew, Shue, Andrew, Eppley, Shue, (, born, ...","[[Andrew, Shue, is, an, American, actor, ,, en...",InstructGPT,open_ended,factscore,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 1...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [0],..."


In [15]:
factscore_origin_path = "../src/data/factscore_origin/labeled"
os.listdir(factscore_origin_path)

['InstructGPT.jsonl',
 'ChatGPT.jsonl',
 'PerplexityAI.jsonl',
 'prompt_entities.txt']

In [16]:
df_chatgpt = pd.read_json(os.path.join(factscore_origin_path, "ChatGPT.jsonl"), lines=True)
df_chatgpt.head()

,input,output,topic,cat,annotations
0,Question: Tell me a bio of Lanny Flaherty.,Lanny Flaherty is an American actor born on De...,Lanny Flaherty,"[very rare, North America]",[{'text': 'Lanny Flaherty is an American actor...
1,Question: Tell me a bio of Marianne McAndrew.,Marianne McAndrew is an American actress and s...,Marianne McAndrew,"[very rare, North America]",[{'text': 'Marianne McAndrew is an American ac...
2,Question: Tell me a bio of Chaim Malinowitz.,"I'm sorry, but I cannot find any reliable info...",Chaim Malinowitz,"[very rare, North America]",None
3,Question: Tell me a bio of Doug Sheehan.,Doug Sheehan is an American actor who was born...,Doug Sheehan,"[very rare, North America]",[{'text': 'Doug Sheehan is an American actor w...
4,Question: Tell me a bio of Gerhard Fischer (in...,Gerhard Fischer is an inventor and entrepreneu...,Gerhard Fischer (inventor),"[very rare, North America]",[{'text': 'Gerhard Fischer is an inventor and ...


In [28]:
df_chatgpt[df_chatgpt["topic"] == "Mike Trivisonno"]

,input,output,topic,cat,annotations
39,Question: Tell me a bio of Mike Trivisonno.,Mike Trivisonno is a well-known radio personal...,Mike Trivisonno,"[rare, North America]",[{'text': 'Mike Trivisonno is a well-known rad...


In [20]:
df_chatgpt.iloc[0]["output"]

'Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida. He has appeared in numerous films, television shows, and theater productions throughout his career, which began in the late 1970s. Some of his notable film credits include "King of New York," "The Abyss," "Natural Born Killers," "The Game," and "The Straight Story." On television, he has appeared in shows such as "Law & Order," "The Sopranos," "Boardwalk Empire," and "The Leftovers." Flaherty has also worked extensively in theater, including productions at the Public Theater and the New York Shakespeare Festival. He is known for his distinctive looks and deep gravelly voice, which have made him a memorable character actor in the industry.'

In [18]:
len(df_chatgpt.iloc[0]["annotations"])

6

In [29]:
np.average(list(chain.from_iterable([[x["label"] == "S" for x in annotation["human-atomic-facts"]] for annotation in df_chatgpt.iloc[39]["annotations"]])))

0.6153846153846154

In [30]:
df_chatgpt.iloc[39]

input                Question: Tell me a bio of Mike Trivisonno.
output         Mike Trivisonno is a well-known radio personal...
topic                                            Mike Trivisonno
cat                                        [rare, North America]
annotations    [{'text': 'Mike Trivisonno is a well-known rad...
Name: 39, dtype: object